# Decorators, list comprechension, contex managers and functional programming

Imports for the lecture

In [1]:
import random
from typing import List, Any, Iterable
from types import FunctionType
from functools import wraps, reduce
from sys import stderr

# Decorators

We've seen examples for decorators in the previous lecture (`@staticmethod`, `@classmethod`, `@property`). Let's see how to write custom decorators!

## Nested functions

Functions can be nested and the child function can only be accessed through the parent.

In [2]:
def parent():
    print("I'm the parent function")
    
    def child():
        print("I'm the child function")
    
    print("Calling the nested function")
    child()
        
parent()
# parent.child  # raises AttributeError

I'm the parent function
Calling the nested function
I'm the child function


Functions can be return values

In [3]:
def parent():
    print("I'm the parent function")
    
    def child():
        print("I'm the child function")
        
    return child

child_func = parent()

print("Calling child")
child_func()


print("\nUsing parent's return value right away")
parent()()

I'm the parent function
Calling child
I'm the child function

Using parent's return value right away
I'm the parent function
I'm the child function


Nested functions have access to the parent's scope

In [4]:
def parent(value: Any):
    
    def child():
        print("I'm the nested function. "
              "The parent's value is {}".format(value))
        
    return child
        
child_func = parent(42)

print("Calling child_func")
child_func()

Calling child_func
I'm the nested function. The parent's value is 42


Function factory

In [5]:
def make_func(param: Any):
    value = param
    
    def func():
        print(f"I'm the nested function. "
               "The parent's value is {value}")
        
    return func

func_11 = make_func(11)
func_abc = make_func("abc")

func_11()
func_abc()


I'm the nested function. The parent's value is {value}
I'm the nested function. The parent's value is {value}



Wrapper function factory
 - let's create a function that takes a function return an almost identical function
 - the returned function adds some logging

In [6]:
def add_noise(func: FunctionType):
    
    def wrapped_with_noise():
        print("Calling function {}".format(func.__name__))
        func()
        print("{} finished.".format(func.__name__))
        
    return wrapped_with_noise

Wrapping a function

In [7]:
def noiseless_function():
    print("This is not noise")

noisy_function = add_noise(noiseless_function)
noisy_function()

Calling function noiseless_function
This is not noise
noiseless_function finished.


## Decorator syntax

 - a decorator is a function
     - that takes a function as an argument
     - returns a wrapped version of the function
 - the decorator syntax is just syntactic sugar (shorthand) for:

In [8]:
@add_noise
def informal_greeter():
    print("Yo")
    
# informal_greeter = add_noise(informal_greeter)
    
informal_greeter()

Calling function informal_greeter
Yo
informal_greeter finished.


## Decorators can take parameters too

They have to return a decorator without parameters - decorator factory

In [9]:
def decorator_with_param(param1: Any, param2: Any=None):
    print("Creating a new decorator: {0}, {1}".format(
        param1, param2))
    def actual_decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            print(f"Wrapper function {func.__name__}")
            print(f"Params: {param1}, {param2}")
            return func(*args, **kwargs)
        return wrapper
    return actual_decorator

In [10]:
@decorator_with_param(42, "abc")
def personal_greeter(name):
    print(f"Hello {name}")
    
@decorator_with_param(4)
def personal_greeter2(name):
    print(f"Hello {name}")
    
print("\nCalling personal_greeter")
personal_greeter("Mary")

Creating a new decorator: 42, abc
Creating a new decorator: 4, None

Calling personal_greeter
Wrapper function personal_greeter
Params: 42, abc
Hello Mary


Decorators can be implemented as classes and `__call__` implements the wrapped function

In [11]:
class MyDecorator(object):
    def __init__(self, func: FunctionType):
        self.func_to_wrap = func
        wraps(func)(self)
        
    def __call__(self, *args, **kwargs):
        print(f"before func {self.func_to_wrap.__name__}")
        res = self.func_to_wrap(*args, **kwargs)
        print(f"after func {self.func_to_wrap.__name__}")
        return res
    
@MyDecorator
def foo():
    print("bar")

foo()

before func foo
bar
after func foo


# List comprehension

- Transform any iterable into a list in one line.
- Syntactic sugar, could be replaced with a for loop.
- Example: create a list of the first N odd numbers starting from 1

In [12]:
l = []
for i in range(10):
    l.append(2*i+1)
l

[1, 3, 5, 7, 9, 11, 13, 15, 17, 19]

One-liner equivalent

In [13]:
l = [2*i+1 for i in range(10)]
l

[1, 3, 5, 7, 9, 11, 13, 15, 17, 19]

The general form of list comprehension is

~~~python
[<expression> for <element> in <sequence>]
~~~

conditional expressions can be added to filter the sequence:

~~~python
[<expression> for <element> in <sequence> if <condition>]
~~~

In [14]:
even = [n*n for n in range(20) if n % 2 == 0]
even

[0, 4, 16, 36, 64, 100, 144, 196, 256, 324]

Which is equivalent to

In [15]:
even = []
for n in range(20):
    if n % 2 == 0:
        even.append(n)
even

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]

- Since this expression implements a filtering mechanism, there is no `else` clause.

- An if-else clause can be used as the first expression though:

In [16]:
l = [1, 0, -2, 3, -1, -5, 0]

signum_l = [int(n / abs(n)) if n != 0 else 0 for n in l]
signum_l

[1, 0, -1, 1, -1, -1, 0]

In [17]:
n = -3.2
int(n / abs(n)) if n != 0 else 0

-1

More than one sequence may be traversed. Is this depth-first or breadth-first traversal?

In [18]:
l1 = [1, 2, 3]
l2 = [4, 5, 6]

[(i, j) for i in l1 for j in l2]

[(1, 4), (1, 5), (1, 6), (2, 4), (2, 5), (2, 6), (3, 4), (3, 5), (3, 6)]

In [19]:
for i in l1:
    for j in l2:
        print((i, j))

(1, 4)
(1, 5)
(1, 6)
(2, 4)
(2, 5)
(2, 6)
(3, 4)
(3, 5)
(3, 6)


List comprehensions may be nested by replacing the first expression with another list comprehension:

In [20]:
matrix = [
    [1, 2, 3],
    [5, 6, 7]
]

[[e*e for e in row] for row in matrix]

[[1, 4, 9], [25, 36, 49]]

But don't go overboard:

In [21]:
[[[random.randint(1, 5) for k in range(3)] for j in range(2)] for i in range(5)]

[[[5, 2, 2], [3, 5, 3]],
 [[5, 2, 5], [5, 5, 3]],
 [[2, 3, 1], [4, 4, 3]],
 [[3, 4, 1], [1, 3, 4]],
 [[4, 2, 5], [3, 1, 1]]]

What is the type of a (list) comprehension?

In [22]:
gen = (i for i in range(10))
type(gen), gen

(generator, <generator object <genexpr> at 0x7fbce548dbd0>)

# Generator expressions

Generator expressions are a generalization of list comprehension. They were introduced in PEP 289 in 2002.

Check out the memory consumption of these cells.

In [23]:
N = 8
s = sum([i*2 for i in range(int(10**N))])
print(s)

9999999900000000


In [24]:
s = sum(i*2 for i in range(int(10**N)))
print(s)

9999999900000000


Generators do not generate a list in memory:

In [25]:
even_numbers = (2*n for n in range(10))
even_numbers

<generator object <genexpr> at 0x7fbce548e110>

Therefore they can only be traversed once:

In [26]:
even_numbers = (2*n for n in range(10))

for num in even_numbers:
    print(num)

0
2
4
6
8
10
12
14
16
18


The generator is empty after the first run:

In [27]:
for num in even_numbers:
    print(num)

Calling `next()` raises a `StopIteration` exception

In [28]:
even_numbers = (2*n for n in range(10))

while True:
    try:
        print(next(even_numbers))
    except StopIteration:
        break

0
2
4
6
8
10
12
14
16
18


In [29]:
# next(even_numbers)  # raises StopIteration

These are actually the defining properties of the **iteration protocol**:

# Iteration protocol

A class satisfies the iteration protocol if:

1. it has a `__iter__` function that returns and iterator, which
1. has a `__next__` function,
2. which raises a `StopIteration` after a certain number of iterations.

For loops use the iteration protocol.

A minimal iterator looks like this:

In [30]:
class MyIterator:
    def __init__(self, iter_no):
        self.iter_no = iter_no
        self._i = iter_no
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if self._i <= 0:
            self._i = self.iter_no
            raise StopIteration()
        self._i -= 1
        print(f"Returning {self._i}")
        return self._i
    
myiter = MyIterator(3)
print("Iterate once")
for i in myiter:
    print(i)
print("Iterate the second time")
for i in myiter:
    print(i)

Iterate once
Returning 2
2
Returning 1
1
Returning 0
0
Iterate the second time
Returning 2
2
Returning 1
1
Returning 0
0


The built-in functions `min`, `max` and `sum` use the iteration protocol:

In [31]:
class AbsoluteNumberContainer:
    def __init__(self, numbers):
        self.numbers = []
        for n in numbers:
            self.numbers.append(abs(n))
        self._i = 0
            
    def __iter__(self):
        # Could be implemented with this line without __next__:
        # return iter(self.numbers)
        return self
    
    def __next__(self):
        if self._i >= len(self.numbers):
            # Reset the loop variable for the next iteration.
            self._i = 0
            raise StopIteration()
        self._i += 1
        return self.numbers[self._i - 1]
    
    
a = AbsoluteNumberContainer([-2, 1, -100])
for n in a:
    print(n)
    
print(f"{max(a) = }\n{min(a) = }\n{sum(a) = }")

2
1
100
max(a) = 100
min(a) = 1
sum(a) = 103


# Set and dict comprehension

Sets and dictionaries can be instantiated via generator expressions too.

A generator expression between curly brackets instantiates a set:

In [32]:
fruit_list = ["apple", "plum", "apple", "pear"]

fruits = {fruit.title() for fruit in fruit_list}

type(fruits), len(fruits), fruits

(set, 3, {'Apple', 'Pear', 'Plum'})

If the expression in the generator is a key-value pair separated by a colon, it instantiates a dictionary:

In [33]:
word_list = ["apple", "plum", "pear", "apple", "apple"]
word_length = {word: len(word) for word in word_list}
type(word_length), len(word_length), word_length

(dict, 3, {'apple': 5, 'plum': 4, 'pear': 4})

In [34]:
word_list = ["apple", "plum", "pear", "avocado"]
first_letters = {word[0]: word for word in word_list}
first_letters

{'a': 'avocado', 'p': 'pear'}

In [35]:
for letter, fruit in first_letters.items():
    print(letter, fruit)

a avocado
p pear


# `yield` keyword

- If a function uses `yield` instead of return, it becomes a **generator function**.
- `yield` temporarily gives back the execution to the caller.
- The generator function continues where it left off after `next` returns:

In [36]:
def german_vowels():
    alphabet = ("a", "ä", "e", "i", "o", "ö", "u", "ü")
    for vowel in alphabet:
        yield vowel

this function returns a generator object

In [37]:
type(german_vowels())

generator

In [38]:
for vowel in german_vowels():
    print(vowel)

a
ä
e
i
o
ö
u
ü


In [39]:
def dummy_generator():
    yield "one"
    yield "two"
    yield "three"
    
for e in dummy_generator():
    print(e)

one
two
three


They can only iterated once:

In [40]:
gen = german_vowels()

print(f"first iteration: {', '.join(gen)}")
print(f"second iteration: {', '.join(gen)}")

first iteration: a, ä, e, i, o, ö, u, ü
second iteration: 


The `next` function returns the next element of the generator.
A `StopIteration` is raised when no more elements are left:

In [41]:
gen = german_vowels()

while True:
    try:
        print(f"The next element is {next(gen)}")
    except StopIteration:
        print("No more elements left :(")
        break

The next element is a
The next element is ä
The next element is e
The next element is i
The next element is o
The next element is ö
The next element is u
The next element is ü
No more elements left :(


The generator function returns a new generator object every time it's called:

In [42]:
gen1 = german_vowels()
gen2 = german_vowels()

print(gen1 is gen2)
print("gen1 first time:", list(gen1))
print("gen1 second time:", list(gen1))
print("gen2 first time:", list(gen2))

False
gen1 first time: ['a', 'ä', 'e', 'i', 'o', 'ö', 'u', 'ü']
gen1 second time: []
gen2 first time: ['a', 'ä', 'e', 'i', 'o', 'ö', 'u', 'ü']


Iterators can only be traversed forward, but we can easily wrap an iterator to have memory:

In [43]:
def iter_with_memory(orig_iter):
    prev = None
    for current in orig_iter:
        yield current, prev
        prev = current

In [44]:
for i in iter_with_memory(german_vowels()):
    print(i)

('a', None)
('ä', 'a')
('e', 'ä')
('i', 'e')
('o', 'i')
('ö', 'o')
('u', 'ö')
('ü', 'u')


## Applications

Generator expressions can be particularly useful for formatted output. We will demonstrate this through a few examples.

In [45]:
numbers = [1, -2, 3, 1]

# print(", ".join(numbers))  # raises TypeError
print(", ".join(str(number) for number in numbers))

1, -2, 3, 1


In [46]:
shopping_list = ["apple", "plum", "pear"]

~~~
The shopping list is:
item 1: apple
item 2: plum
item 3: pear
~~~

In [47]:
shopping_list = ["apple", "plum", "pear"]

print("The shopping list is:\n{}".format(
    "\n".join(
        f"item {idx+1}: {element}"
        for idx, element in enumerate(shopping_list))
))

The shopping list is:
item 1: apple
item 2: plum
item 3: pear


__Print the following shopping list with quantities.__

For example:

~~~
item 1: apple, quantity: 2
item 2: pear, quantity: 1
~~~

In [48]:
shopping_list = {
    "apple": 2,
    "pear": 1,
    "plum": 5,
}
# TODO
print(
    "\n".join(
        f"item {idx+1}: {element}, quantity: {quantity}"
        for idx, (element, quantity) in enumerate(shopping_list.items()))
)

item 1: apple, quantity: 2
item 2: pear, quantity: 1
item 3: plum, quantity: 5


__Print the same format in alphabetical order.__

Decreasing order by quantity:

In [49]:
shopping_list = {
    "apple": 2,
    "pear": 1,
    "plum": 5,
}
# TODO

# Context managers

There are two types of resources: managed and unmanaged.

__Managed resources__

- Resource acquisition and release are automatically done.
- No need for manual resource management.
- Example: memory

__Unmanaged resources__

- Unmanaged resources need explicit release.
- Otherwise the operating system may run out of the resource.
- Examples include files, network sockets.


In [50]:
fh = []
while True:
    try:
        fh.append(open("abc.txt", "w"))
    except OSError:
        break
len(fh)

1048519

We can't open more files:

In [51]:
fh2 = []
while True:
    try:
        fh2.append(open("abc.txt", "w"))
    except OSError:
        break
len(fh), len(fh2)

(1048519, 0)

Exception in thread Exception in threading.excepthook:
Exception ignored in thread started by: <bound method Thread._bootstrap of <HistorySavingThread(IPythonHistorySavingThread, started 140449306105408)>>
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 966, in _bootstrap
  File "/usr/lib/python3.10/threading.py", line 1011, in _bootstrap_inner


The history saving thread hit an unexpected error (OperationalError('unable to open database file')).History will not be written to the database.

  File "/usr/lib/python3.10/threading.py", line 1329, in invoke_excepthook
zmq.error.ZMQError: Too many open files
Exception ignored in sys.unraisablehook: <built-in function unraisablehook>
Traceback (most recent call last):
  File "/home/kinga/.local/lib/python3.10/site-packages/ipykernel/iostream.py", line 475, in flush
  File "/home/kinga/.local/lib/python3.10/site-packages/ipykernel/iostream.py", line 210, in schedule
zmq.error.ZMQError: Too many open files


In [52]:
for f in fh:
    f.close()

- We need to manually close the file.
- What happens when an exception occurs?

In [53]:
s1 = "important text"
fh = open("file.txt", "w")
# fh.write(s2)  # raises NameError
fh.close()

- The file is never closed, the file descriptor **is leaked**.
- A solution would be to use try-except blocks with `finally` clauses.

In [54]:
fh = open("file.txt", "w")
try:
    fh.write(important_variable)
except Exception as e:
    stderr.write(f"{type(e).__name__} happened")
finally:
    print("Closing file")
    fh.close()

Closing file


NameError happened

__Context managers handle this automatically__

- The `with` keyword opens a resource,
- keeps it open until the execution leaves with's scope,
- releases the resource regardless whether an exception is raised or not.

In [55]:
with open("file.txt", "w") as fh:
    fh.write("abc\n")
    # fh.write(important_variable)  # raises NameError

`file.txt` is no longer open:

In [56]:
# fh.write("ab") # raises ValueError: I/O operation on closed file.

## Defining context managers

Any class can be a context manager if it implements:
  1. `__enter__`: runs at the beginning of the `with`. Returns the resource.
  1. `__exit__`: runs after the with block. Releases the resource.

In [57]:
class DummyContextManager:
    def __init__(self, value):
        self.value = value
        
    def __enter__(self):
        print("Dummy resource acquired")
        return self.value
    
    def __exit__(self, *args):
        print("Dummy resource released")
        
with DummyContextManager(42) as d:
    print(f"Resource: {d}")

Dummy resource acquired
Resource: 42
Dummy resource released


`__exit__` takes 3 extra arguments that describe the exception: `exc_type`, `exc_value`, `traceback`

In [58]:
class DummyContextManager:
    def __init__(self, value):
        self.value = value
        
    def __enter__(self):
        print("Dummy resource acquired")
        return self.value
    
    def __exit__(self, exc_type, exc_value, traceback):
        if exc_type is not None:
            print(f"{exc_type} with value {exc_value} caught\nTraceback: {traceback}")
        print("Dummy resource released")
        
with DummyContextManager(42) as d:
    print(d)
    # raise ValueError("just because I can")  # __exit__ will be called anyway

Dummy resource acquired
42
Dummy resource released


# Functional Python: map, filter and reduce

Python has a few built-in functions that originate from functional programming.


## Map

`map` applies a _callable_ on each element of a sequence.

This can be a function:

In [59]:
def double(e: Any):
    return e * 2

l = [2, 3, "abc"]

list(map(double, l))

[4, 6, 'abcabc']

In [60]:
map(double, l)

A `lambda` expression:

In [61]:
list(map(lambda x: x * 2, [2, 3, "abc"]))

[4, 6, 'abcabc']

Or a class:

In [62]:
class Doubler:
    def __call__(self, v: Any):
        return v * 2

doubler_instance = Doubler()

list(map(doubler_instance, l))

[4, 6, 'abcabc']

In [63]:
class Multiplier:
    def __init__(self, k: Any):
        self.k = k
        
    def __call__(self, v: float | int):
        return v * self.k
    
doubler = Multiplier(2)
tripler = Multiplier(3)

It's evaluated in a lazy fashion. The return type is an iterable:

In [64]:
map(double, l)

In [65]:
class Doubler:
    def __call__(self, v: Any):
        print(f"Doubling {v}")
        return v * 2

doubler_instance = Doubler()

mapped_l = map(doubler_instance, l)
mapped_l

The actual doubling is only done when its result is needed:

In [66]:
list(mapped_l)

Doubling 2
Doubling 3
Doubling abc


[4, 6, 'abcabc']

The iterator is _empty_ now:

In [67]:
list(mapped_l)

[]

## Filter

Filter creates a list of elements for which a function returns true.

In [68]:
def is_even(n: int):
    return n % 2 == 0

l = [2, 3, -1, 0, 2]

list(filter(is_even, l))

[2, 0, 2]

In [69]:
list(filter(lambda x: x % 2 == 0, range(8)))

[0, 2, 4, 6]

In [70]:
r = range(12)
r, list(r)

(range(0, 12), [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11])

In [71]:
list(r)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

Most comprehensions can be rewritten using `map` and `filter`:

In [72]:
l = [2, 3, -1, 0, 2]

[x for x in l if x % 2 == 0]

[2, 0, 2]

Signum example:

In [73]:
l = [2, 3, 0, -1, 2, 0, 1]

signum = [x / abs(x) if x != 0 else x for x in l]
print(signum)

[1.0, 1.0, 0, -1.0, 1.0, 0, 1.0]


In [74]:
list(map(lambda x: x / abs(x) if x != 0 else 0, l))

[1.0, 1.0, 0, -1.0, 1.0, 0, 1.0]

## Zip

`zip` pairs elements of two iterables:

In [75]:
l1 = ["apple", "plum", "pear"]
l2 = [10, 2, 3]

for elements in zip(l1, l2):
    print(elements)

('apple', 10)
('plum', 2)
('pear', 3)


They can have different length:

In [76]:
l1 = ["apple", "plum", "pear"]
l2 = [10, 2, 3, -1, -2]

for fruit, quantity in zip(l1, l2):
    print(fruit, quantity)

apple 10
plum 2
pear 3


More generally `zip` transposes a list of iterables:

In [77]:
row1 = [1, 2, 3, 4]
row2 = [1, 2, 3, 4]
row3 = [-1, -2, -3, -4]

for column in zip(row1, row2, row3):
    print(column)

(1, 1, -1)
(2, 2, -2)
(3, 3, -3)
(4, 4, -4)


We can implement matrix transpose with `zip`:

In [78]:
def transpose(mtx):
    return list(map(list, zip(*mtx)))
    # OR
    # return [list(col) for col in zip(*mtx)]


mtx = [[1, 2, 3], [4, 5, 6]]

transpose(mtx)

[[1, 4], [2, 5], [3, 6]]

In [79]:
row1 = [1, 2, 3, 4]
row2 = [1, 2, 3, 4]
row3 = [-1, -2, -3, -4]
z = zip(row1, row2, row2)
z, type(z)

(<zip at 0x7fbce5e24f80>, zip)

In [80]:
list(z)

[(1, 1, 1), (2, 2, 2), (3, 3, 3), (4, 4, 4)]

In [81]:
# next(z)  # raises StopIteration

## Reduce

- Reduce applies a rolling computation on a sequence.
- The first argument of `reduce` is two-argument function.
- The second argument is the sequence.
- The result is accumulated in an accumulator.

In [82]:
l = [1, 2, -1, 4]
reduce(lambda x, y: x*y, l)

-8

An initial value for the accumulator may be supplied:

In [83]:
reduce(lambda x, y: x*y, l, 10)

-80

Finding the maximum with reduce:

In [84]:
reduce(lambda x, y: max(x, y), l)

4

Same with the built-in function:

In [85]:
reduce(max, l)

4

Summing even numbers only:

In [86]:
l = [1, 2, -1, 4]
reduce(lambda x, y: x + int(y % 2 == 0) * y, l, 0)

6

Booleans can be summed:

In [87]:
sum(e % 2 == 0 for e in l)

2

For historical reasons, they are actually integers:

In [88]:
int(True), int(False), isinstance(False, int), isinstance(True, int)

(1, 0, True, True)

# Recommended reading

- [Itertools](https://docs.python.org/3.8/library/itertools.html) is a collection of iteration related building blocks.
- [Functools](https://docs.python.org/3.8/library/functools.html) is a module for higher order functions.